# MLOps Pipeline Workflow & Team Notes

## Project Overview
This notebook implements an end-to-end MLOps data pipeline using the **Olist Brazilian E-Commerce dataset**. The goal was to demonstrate a production-style workflow that covers data ingestion, cataloging, exploratory analysis, feature engineering, feature storage, and dataset splitting — all using AWS services in a cost-efficient way.

The pipeline was intentionally built step-by-step to mirror MLOps practices rather than a one-off modeling notebook.

---

## High-Level Workflow

### 1. Raw Data Ingestion (S3 Data Lake)
- Created an S3 bucket to act as a data lake.
- Uploaded all **9 raw CSV files** from the Kaggle Olist dataset.
- Organized raw data under: s3:///raw/olist/ingest_date=YYYY-MM-DD/
- Each dataset was placed into its own subfolder to support Athena’s directory-based table requirements.

---

### 2. Data Cataloging & Querying (Athena)
- Created an Athena database (`olist_datalake`).
- Defined **external tables** for each dataset directly from JupyterLab (no Glue crawler required).
- Verified schemas and row counts using Athena queries.
- This step enabled SQL-based access to the data and served as the cataloging layer for downstream analysis.

---

### 3. Exploratory Data Analysis (SageMaker + Pandas)
- Loaded Athena tables into Pandas using `awswrangler`.
- Performed sanity checks on row counts and joins.
- Built an **order-level analytical view** by aggregating:
- order items
- payments
- customer attributes
- Engineered a target variable (`is_late`) based on delivery vs. estimated delivery dates.
- Observed class imbalance (~8% late deliveries), motivating careful splitting and evaluation later.

---

### 4. Feature Engineering
- Created leakage-safe, order-level features using only information available at purchase time:
- pricing, freight, number of items/sellers
- payment information
- time-based features (day of week, hour of day)
- customer state
- Maintained a **canonical feature dataset** for analysis and splitting.
- Created a **Feature Store–compatible version** with strict data types.

---

### 5. SageMaker Feature Store (Offline Store)
- Created a **SageMaker Feature Group** (offline store only to control cost).
- Used `order_id` as the record identifier.
- Used a strictly formatted ISO-8601 `event_time` with UTC (`Z`) as the event time feature.
- Successfully ingested ~99k feature records into Feature Store.
- Offline store data is persisted in S3 for training and future reuse.

---

### 6. Dataset Splitting (Time-Based)
- Performed a **time-based split** using `event_time` to avoid temporal leakage:
- Train: ~40%
- Validation: ~10%
- Test: ~10%
- Production reserve: ~40%
- Persisted each split as Parquet files to: s3:///splits/olist/features/version=v1/
- This mirrors a real production setup where recent data is reserved for inference.

---

## Key Engineering Decisions & Lessons Learned

- **Athena LOCATION must point to directories**, not individual files.
- **Feature Store requires strict ISO-8601 timestamps with timezone** — missing the `Z` suffix causes ingestion failures.
- Maintaining separate:
- canonical feature data (analysis-friendly)
- Feature Store–safe data (schema-restricted)
is a best practice in real MLOps systems.
- Time-based splitting is critical to avoid data leakage in temporal datasets.
- Offline Feature Store provides the required functionality while minimizing cost.

---

## Cost Management Notes
- SageMaker compute was stopped immediately after completion.
- Feature Store **online store was intentionally disabled** to avoid ongoing charges.
- S3 storage costs are minimal and safe to keep until final submission.
- Cleanup (Feature Group deletion, S3 cleanup) should only be done **after submission**.

---

## For Teammates
If you need to re-run or extend this work:
1. Start at the Athena read step (no need to re-upload raw data).
2. Do **not** re-run ingestion unless changing the feature schema.
3. Always stop SageMaker compute when finished.

This notebook represents a complete, MLOps data pipeline.



In [1]:
import boto3

bucket = "aai540-olist-mlops-chris-7f3k2p"
prefix = "raw/olist/ingest_date=2026-01-25/"

s3 = boto3.client("s3")
resp = s3.list_objects_v2(Bucket=bucket, Prefix=prefix)

keys = [obj["Key"] for obj in resp.get("Contents", [])]
print("Found files:", len(keys))
for k in keys:
    print(k)

Found files: 10
raw/olist/ingest_date=2026-01-25/
raw/olist/ingest_date=2026-01-25/olist_customers_dataset.csv
raw/olist/ingest_date=2026-01-25/olist_geolocation_dataset.csv
raw/olist/ingest_date=2026-01-25/olist_order_items_dataset.csv
raw/olist/ingest_date=2026-01-25/olist_order_payments_dataset.csv
raw/olist/ingest_date=2026-01-25/olist_order_reviews_dataset.csv
raw/olist/ingest_date=2026-01-25/olist_orders_dataset.csv
raw/olist/ingest_date=2026-01-25/olist_products_dataset.csv
raw/olist/ingest_date=2026-01-25/olist_sellers_dataset.csv
raw/olist/ingest_date=2026-01-25/product_category_name_translation.csv


In [2]:
import boto3

bucket = "aai540-olist-mlops-chris-7f3k2p"
results_prefix = "athena-results/"

s3 = boto3.client("s3")
resp = s3.list_objects_v2(Bucket=bucket, Prefix=results_prefix, MaxKeys=1)

if "Contents" in resp:
    print("✅ Athena results prefix exists:", f"s3://{bucket}/{results_prefix}")
else:
    # create a zero-byte object so the prefix exists
    s3.put_object(Bucket=bucket, Key=results_prefix)
    print("✅ Created Athena results prefix:", f"s3://{bucket}/{results_prefix}")

✅ Athena results prefix exists: s3://aai540-olist-mlops-chris-7f3k2p/athena-results/


In [3]:
import time
import boto3

bucket = "aai540-olist-mlops-chris-7f3k2p"
REGION = boto3.session.Session().region_name
athena = boto3.client("athena", region_name=REGION)

ATHENA_OUTPUT = f"s3://{bucket}/athena-results/"
DB = "olist_datalake"

def run_athena(sql: str, database: str = "default"):
    res = athena.start_query_execution(
        QueryString=sql,
        QueryExecutionContext={"Database": database},
        ResultConfiguration={"OutputLocation": ATHENA_OUTPUT},
    )
    qid = res["QueryExecutionId"]
    while True:
        q = athena.get_query_execution(QueryExecutionId=qid)
        state = q["QueryExecution"]["Status"]["State"]
        if state in ("SUCCEEDED", "FAILED", "CANCELLED"):
            break
        time.sleep(1)
    if state != "SUCCEEDED":
        reason = q["QueryExecution"]["Status"].get("StateChangeReason", "Unknown")
        raise RuntimeError(f"Athena query failed: {state} - {reason}\nSQL:\n{sql}")
    return qid

run_athena(f"CREATE DATABASE IF NOT EXISTS {DB};", database="default")
print("✅ Database ready:", DB)

✅ Database ready: olist_datalake


In [5]:
import boto3

bucket = "aai540-olist-mlops-chris-7f3k2p"
base_prefix = "raw/olist/ingest_date=2026-01-25/"

files = [
    "olist_customers_dataset.csv",
    "olist_geolocation_dataset.csv",
    "olist_order_items_dataset.csv",
    "olist_order_payments_dataset.csv",
    "olist_order_reviews_dataset.csv",
    "olist_orders_dataset.csv",
    "olist_products_dataset.csv",
    "olist_sellers_dataset.csv",
    "product_category_name_translation.csv",
]

s3 = boto3.client("s3")

for f in files:
    src_key = base_prefix + f
    folder = f.replace(".csv", "")  # folder name = file name without .csv
    dst_key = f"{base_prefix}{folder}/{f}"
    
    # copy
    s3.copy_object(
        Bucket=bucket,
        CopySource={"Bucket": bucket, "Key": src_key},
        Key=dst_key
    )
    print("✅ Copied to:", dst_key)

print("\nDone. Next we’ll point Athena tables at these folders.")

✅ Copied to: raw/olist/ingest_date=2026-01-25/olist_customers_dataset/olist_customers_dataset.csv
✅ Copied to: raw/olist/ingest_date=2026-01-25/olist_geolocation_dataset/olist_geolocation_dataset.csv
✅ Copied to: raw/olist/ingest_date=2026-01-25/olist_order_items_dataset/olist_order_items_dataset.csv
✅ Copied to: raw/olist/ingest_date=2026-01-25/olist_order_payments_dataset/olist_order_payments_dataset.csv
✅ Copied to: raw/olist/ingest_date=2026-01-25/olist_order_reviews_dataset/olist_order_reviews_dataset.csv
✅ Copied to: raw/olist/ingest_date=2026-01-25/olist_orders_dataset/olist_orders_dataset.csv
✅ Copied to: raw/olist/ingest_date=2026-01-25/olist_products_dataset/olist_products_dataset.csv
✅ Copied to: raw/olist/ingest_date=2026-01-25/olist_sellers_dataset/olist_sellers_dataset.csv
✅ Copied to: raw/olist/ingest_date=2026-01-25/product_category_name_translation/product_category_name_translation.csv

Done. Next we’ll point Athena tables at these folders.


In [6]:
resp = s3.list_objects_v2(Bucket=bucket, Prefix=base_prefix, MaxKeys=50)
for obj in resp.get("Contents", []):
    print(obj["Key"])

raw/olist/ingest_date=2026-01-25/
raw/olist/ingest_date=2026-01-25/olist_customers_dataset.csv
raw/olist/ingest_date=2026-01-25/olist_customers_dataset/olist_customers_dataset.csv
raw/olist/ingest_date=2026-01-25/olist_geolocation_dataset.csv
raw/olist/ingest_date=2026-01-25/olist_geolocation_dataset/olist_geolocation_dataset.csv
raw/olist/ingest_date=2026-01-25/olist_order_items_dataset.csv
raw/olist/ingest_date=2026-01-25/olist_order_items_dataset/olist_order_items_dataset.csv
raw/olist/ingest_date=2026-01-25/olist_order_payments_dataset.csv
raw/olist/ingest_date=2026-01-25/olist_order_payments_dataset/olist_order_payments_dataset.csv
raw/olist/ingest_date=2026-01-25/olist_order_reviews_dataset.csv
raw/olist/ingest_date=2026-01-25/olist_order_reviews_dataset/olist_order_reviews_dataset.csv
raw/olist/ingest_date=2026-01-25/olist_orders_dataset.csv
raw/olist/ingest_date=2026-01-25/olist_orders_dataset/olist_orders_dataset.csv
raw/olist/ingest_date=2026-01-25/olist_products_dataset.csv


In [7]:
RAW_BASE = f"s3://{bucket}/{base_prefix}"

def create_csv_table(table_name: str, columns_ddl: str, folder_name: str):
    sql = f"""
    CREATE EXTERNAL TABLE IF NOT EXISTS {DB}.{table_name} (
      {columns_ddl}
    )
    ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde'
    WITH SERDEPROPERTIES (
      'separatorChar' = ',',
      'quoteChar'     = '\"',
      'escapeChar'    = '\\\\'
    )
    STORED AS TEXTFILE
    LOCATION '{RAW_BASE}{folder_name}/'
    TBLPROPERTIES ('skip.header.line.count'='1');
    """
    run_athena(sql, database=DB)
    print(f"✅ Created: {DB}.{table_name}")

create_csv_table(
    "olist_customers_dataset",
    """
    customer_id string,
    customer_unique_id string,
    customer_zip_code_prefix int,
    customer_city string,
    customer_state string
    """,
    "olist_customers_dataset"
)

create_csv_table(
    "olist_geolocation_dataset",
    """
    geolocation_zip_code_prefix int,
    geolocation_lat double,
    geolocation_lng double,
    geolocation_city string,
    geolocation_state string
    """,
    "olist_geolocation_dataset"
)

create_csv_table(
    "olist_order_items_dataset",
    """
    order_id string,
    order_item_id int,
    product_id string,
    seller_id string,
    shipping_limit_date string,
    price double,
    freight_value double
    """,
    "olist_order_items_dataset"
)

create_csv_table(
    "olist_order_payments_dataset",
    """
    order_id string,
    payment_sequential int,
    payment_type string,
    payment_installments int,
    payment_value double
    """,
    "olist_order_payments_dataset"
)

create_csv_table(
    "olist_order_reviews_dataset",
    """
    review_id string,
    order_id string,
    review_score int,
    review_comment_title string,
    review_comment_message string,
    review_creation_date string,
    review_answer_timestamp string
    """,
    "olist_order_reviews_dataset"
)

create_csv_table(
    "olist_orders_dataset",
    """
    order_id string,
    customer_id string,
    order_status string,
    order_purchase_timestamp string,
    order_approved_at string,
    order_delivered_carrier_date string,
    order_delivered_customer_date string,
    order_estimated_delivery_date string
    """,
    "olist_orders_dataset"
)

create_csv_table(
    "olist_products_dataset",
    """
    product_id string,
    product_category_name string,
    product_name_lenght int,
    product_description_lenght int,
    product_photos_qty int,
    product_weight_g int,
    product_length_cm int,
    product_height_cm int,
    product_width_cm int
    """,
    "olist_products_dataset"
)

create_csv_table(
    "olist_sellers_dataset",
    """
    seller_id string,
    seller_zip_code_prefix int,
    seller_city string,
    seller_state string
    """,
    "olist_sellers_dataset"
)

create_csv_table(
    "product_category_name_translation",
    """
    product_category_name string,
    product_category_name_english string
    """,
    "product_category_name_translation"
)

✅ Created: olist_datalake.olist_customers_dataset
✅ Created: olist_datalake.olist_geolocation_dataset
✅ Created: olist_datalake.olist_order_items_dataset
✅ Created: olist_datalake.olist_order_payments_dataset
✅ Created: olist_datalake.olist_order_reviews_dataset
✅ Created: olist_datalake.olist_orders_dataset
✅ Created: olist_datalake.olist_products_dataset
✅ Created: olist_datalake.olist_sellers_dataset
✅ Created: olist_datalake.product_category_name_translation


In [8]:
run_athena(f"SHOW TABLES IN {DB};", database=DB)
print("✅ SHOW TABLES succeeded")

run_athena(f"SELECT COUNT(*) FROM {DB}.olist_orders_dataset;", database=DB)
print("✅ COUNT orders succeeded")

run_athena(f"SELECT order_status, COUNT(*) c FROM {DB}.olist_orders_dataset GROUP BY 1 ORDER BY c DESC;", database=DB)
print("✅ GROUP BY order_status succeeded")

✅ SHOW TABLES succeeded
✅ COUNT orders succeeded
✅ GROUP BY order_status succeeded


In [1]:
#6.1
import awswrangler as wr
import pandas as pd

DB = "olist_datalake"

orders = wr.athena.read_sql_query(
    sql=f"SELECT * FROM {DB}.olist_orders_dataset",
    database=DB,
    ctas_approach=False
)

order_items = wr.athena.read_sql_query(
    sql=f"SELECT * FROM {DB}.olist_order_items_dataset",
    database=DB,
    ctas_approach=False
)

payments = wr.athena.read_sql_query(
    sql=f"SELECT * FROM {DB}.olist_order_payments_dataset",
    database=DB,
    ctas_approach=False
)

customers = wr.athena.read_sql_query(
    sql=f"SELECT * FROM {DB}.olist_customers_dataset",
    database=DB,
    ctas_approach=False
)

print("orders:", orders.shape)
print("order_items:", order_items.shape)
print("payments:", payments.shape)
print("customers:", customers.shape)

2026-01-25 19:16:46,584	WARNING services.py:2070 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 1909432320 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=4.64gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2026-01-25 19:16:46,747	INFO worker.py:1852 -- Started a local Ray instance.


orders: (99441, 8)
order_items: (112650, 7)
payments: (103886, 5)
customers: (99441, 5)


In [2]:
#6.2
# Parse timestamps
timestamp_cols = [
    "order_purchase_timestamp",
    "order_approved_at",
    "order_delivered_carrier_date",
    "order_delivered_customer_date",
    "order_estimated_delivery_date",
]
for col in timestamp_cols:
    orders[col] = pd.to_datetime(orders[col], errors="coerce")

# Aggregations
items_agg = (
    order_items.groupby("order_id")
    .agg(
        num_items=("order_item_id", "count"),
        total_price=("price", "sum"),
        total_freight_value=("freight_value", "sum"),
        num_sellers=("seller_id", "nunique"),
    )
    .reset_index()
)

payments_agg = (
    payments.groupby("order_id")
    .agg(
        payment_value=("payment_value", "sum"),
        payment_installments=("payment_installments", "max"),
        payment_type=("payment_type", lambda x: x.value_counts().index[0]),
    )
    .reset_index()
)

eda_df = (
    orders
    .merge(items_agg, on="order_id", how="left")
    .merge(payments_agg, on="order_id", how="left")
    .merge(customers[["customer_id", "customer_state"]], on="customer_id", how="left")
)

# Time features
eda_df["purchase_dow"] = eda_df["order_purchase_timestamp"].dt.dayofweek
eda_df["purchase_hour"] = eda_df["order_purchase_timestamp"].dt.hour

# Label: late delivery
eda_df["is_late"] = (
    (eda_df["order_delivered_customer_date"].notna()) &
    (eda_df["order_estimated_delivery_date"].notna()) &
    (eda_df["order_delivered_customer_date"] > eda_df["order_estimated_delivery_date"])
).astype(int)

print("Orders rows:", len(orders))
print("EDA rows:", len(eda_df))
print("Row loss:", len(orders) - len(eda_df))
print("Late rate:\n", eda_df["is_late"].value_counts(normalize=True))

Orders rows: 99441
EDA rows: 99441
Row loss: 0
Late rate:
 is_late
0    0.92129
1    0.07871
Name: proportion, dtype: float64


In [3]:
#7.0 + 7B
# Canonical features (with purchase timestamp)
feat = eda_df[[
    "order_id",
    "order_purchase_timestamp",
    "customer_state",
    "num_items",
    "total_price",
    "total_freight_value",
    "num_sellers",
    "payment_value",
    "payment_installments",
    "payment_type",
    "purchase_dow",
    "purchase_hour",
    "is_late"
]].copy()

feat["customer_state"] = feat["customer_state"].fillna("unknown").astype(str)
feat["payment_type"] = feat["payment_type"].fillna("unknown").astype(str)

for c in ["num_items", "num_sellers", "payment_installments", "purchase_dow", "purchase_hour", "is_late"]:
    feat[c] = feat[c].fillna(0).astype(int)

for c in ["total_price", "total_freight_value", "payment_value"]:
    feat[c] = feat[c].fillna(0.0).astype(float)

# Create strict ISO-8601 event time WITH timezone "Z"
feat["event_time"] = (
    pd.to_datetime(feat["order_purchase_timestamp"], errors="coerce")
    .dt.strftime("%Y-%m-%dT%H:%M:%SZ")
)

feat = feat.dropna(subset=["order_id", "event_time"]).reset_index(drop=True)

# FeatureStore-safe version (remove datetime64 column)
feat_fs = feat.drop(columns=["order_purchase_timestamp"]).copy()
feat_fs["event_time"] = feat_fs["event_time"].astype(str)

print("✅ feat shape:", feat.shape)
print("✅ feat_fs shape:", feat_fs.shape)
print("event_time sample:", feat_fs["event_time"].head().tolist())

✅ feat shape: (99441, 14)
✅ feat_fs shape: (99441, 13)
event_time sample: ['2017-10-02T10:56:33Z', '2018-07-24T20:41:37Z', '2018-08-08T08:38:49Z', '2017-11-18T19:28:06Z', '2018-02-13T21:18:39Z']


In [4]:
#7.1
import boto3
import sagemaker
from sagemaker.feature_store.feature_group import FeatureGroup

bucket = "aai540-olist-mlops-chris-7f3k2p"
region = boto3.session.Session().region_name
sess = sagemaker.Session()
role = sagemaker.get_execution_role()

feature_group_name = "olist-order-features-v1"
offline_store_s3_uri = f"s3://{bucket}/feature-store/{feature_group_name}/"

fg = FeatureGroup(name=feature_group_name, sagemaker_session=sess)

print("Region:", region)
print("Role:", role)
print("Offline store URI:", offline_store_s3_uri)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
Region: us-east-1
Role: arn:aws:iam::758289042916:role/LabRole
Offline store URI: s3://aai540-olist-mlops-chris-7f3k2p/feature-store/olist-order-features-v1/


In [5]:
#7.2
import time
from botocore.exceptions import ClientError

sm = boto3.client("sagemaker", region_name=region)

def feature_group_exists(name: str) -> bool:
    try:
        sm.describe_feature_group(FeatureGroupName=name)
        return True
    except ClientError as e:
        if "ResourceNotFound" in str(e):
            return False
        raise

def wait_for_fg_created(name: str, timeout_sec: int = 600, poll_sec: int = 10):
    start = time.time()
    while True:
        desc = sm.describe_feature_group(FeatureGroupName=name)
        status = desc.get("FeatureGroupStatus")
        offline_status = desc.get("OfflineStoreStatus", {}).get("Status", "UNKNOWN")
        print(f"Status={status}, OfflineStoreStatus={offline_status}")
        if status == "Created" and offline_status in ("Active", "UNKNOWN"):
            return desc
        if status in ("CreateFailed", "DeleteFailed"):
            raise RuntimeError(f"Feature Group failed with status={status}. Details: {desc}")
        if time.time() - start > timeout_sec:
            raise TimeoutError(f"Timed out waiting for Feature Group to be Created: {name}")
        time.sleep(poll_sec)

if feature_group_exists(feature_group_name):
    print(f"✅ Feature Group already exists: {feature_group_name}")
else:
    fg.load_feature_definitions(data_frame=feat_fs)
    fg.create(
        s3_uri=offline_store_s3_uri,
        record_identifier_name="order_id",
        event_time_feature_name="event_time",
        role_arn=role,
        enable_online_store=False,
    )
    print("⏳ Create request submitted")

wait_for_fg_created(feature_group_name)
print("✅ Feature Group ready")

✅ Feature Group already exists: olist-order-features-v1
Status=Created, OfflineStoreStatus=UNKNOWN
✅ Feature Group ready


In [6]:
#7.3
ingest_response = fg.ingest(data_frame=feat_fs, max_workers=2, wait=True)
print("✅ Ingest complete")

✅ Ingest complete


In [7]:
#7.4
import boto3

sm = boto3.client("sagemaker", region_name=region)
desc = sm.describe_feature_group(FeatureGroupName=feature_group_name)

print("FeatureGroupStatus:", desc["FeatureGroupStatus"])
print("OfflineStoreStatus:", desc["OfflineStoreStatus"]["Status"])
print("S3 Offline Store Uri:", desc["OfflineStoreConfig"]["S3StorageConfig"]["S3Uri"])

FeatureGroupStatus: Created
OfflineStoreStatus: Active
S3 Offline Store Uri: s3://aai540-olist-mlops-chris-7f3k2p/feature-store/olist-order-features-v1/


In [8]:
#8.0
import awswrangler as wr

bucket = "aai540-olist-mlops-chris-7f3k2p"

# Use feat_fs for splits (Feature Store compatible)
feat_sorted = feat_fs.sort_values("event_time").reset_index(drop=True)
n = len(feat_sorted)

train_end = int(n * 0.40)
val_end   = int(n * 0.50)
test_end  = int(n * 0.60)

train_df = feat_sorted.iloc[:train_end]
val_df   = feat_sorted.iloc[train_end:val_end]
test_df  = feat_sorted.iloc[val_end:test_end]
prod_df  = feat_sorted.iloc[test_end:]

print("✅ Split sizes")
print("train:", len(train_df))
print("val:  ", len(val_df))
print("test: ", len(test_df))
print("prod: ", len(prod_df))

split_base = f"s3://{bucket}/splits/olist/features/version=v1/"
wr.s3.to_parquet(train_df, f"{split_base}train/", dataset=True, mode="overwrite")
wr.s3.to_parquet(val_df,   f"{split_base}val/",   dataset=True, mode="overwrite")
wr.s3.to_parquet(test_df,  f"{split_base}test/",  dataset=True, mode="overwrite")
wr.s3.to_parquet(prod_df,  f"{split_base}prod/",  dataset=True, mode="overwrite")

print("✅ Wrote splits to:", split_base)

✅ Split sizes
train: 39776
val:   9944
test:  9944
prod:  39777
✅ Wrote splits to: s3://aai540-olist-mlops-chris-7f3k2p/splits/olist/features/version=v1/


In [9]:
#9.0

In [10]:
import boto3

s3 = boto3.client("s3")
prefix = "splits/olist/features/version=v1/"

resp = s3.list_objects_v2(Bucket=bucket, Prefix=prefix, MaxKeys=50)
print("Objects found:", resp.get("KeyCount", 0))
for obj in resp.get("Contents", [])[:20]:
    print(obj["Key"])

Objects found: 4
splits/olist/features/version=v1/prod/a8cdd289b48b495ba34f072d8dfa9932.snappy.parquet
splits/olist/features/version=v1/test/ba691db52493437ab3c663c8b98fe955.snappy.parquet
splits/olist/features/version=v1/train/33879ef6e41a44e384d5255caa4ffa7f.snappy.parquet
splits/olist/features/version=v1/val/6fb7a4263ec04acb8f57f8ad0050cfb0.snappy.parquet
